In [3]:
! pip install python-gdcm
! pip install pylibjpeg pylibjpeg-libjpeg pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 15.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 4.0 MB/s eta 0:00:0000:0100:010m


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pydicom
import os
import nibabel as nib
import SimpleITK as sitk
from tqdm import tqdm
from tensorflow import keras
from keras import layers, models
import tensorflow_io as tfio
import cv2
import warnings
warnings.filterwarnings('ignore')
from skimage.transform import resize
from tqdm import tqdm
import tensorflow_addons as tfa

In [5]:
def load_image(path):
    img = pydicom.read_file(path)
    img = img.pixel_array
    return cv2.resize(img, (160,160))

In [6]:
def window(img):
    img[np.where(img<400)] = 400
    img[np.where(img>1900)] = 1900
    return (img-400)/1500.0

In [7]:
def channeling(img):
#     std = img.std()
#     mean = img.mean()
#     img1 = img.copy()
#     img2 = img.copy()
    
#     img1[np.where(img1<(mean-std))] = (mean-std)
#     img1[np.where(img1>(mean))] = mean
    
#     img2[np.where(img2<(mean))] = mean
#     img2[np.where(img2>(mean+std))] = (mean+std)
    
#     img[np.where(img<(mean+std))]= (mean+std)
    
    return np.stack([img, img, img], axis=-1)

In [8]:
bb_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train_bounding_boxes.csv")
bb_df.head(30)

,StudyInstanceUID,x,y,width,height,slice_number
0,1.2.826.0.1.3680043.10051,219.27715,216.71419,17.30440,20.38517,133
1,1.2.826.0.1.3680043.10051,221.56460,216.71419,17.87844,25.24362,134
2,1.2.826.0.1.3680043.10051,216.82151,221.62546,27.00959,26.37454,135
3,1.2.826.0.1.3680043.10051,214.49455,215.48637,27.92726,37.51363,136
4,1.2.826.0.1.3680043.10051,214.00000,215.48637,27.00000,43.51363,137
5,1.2.826.0.1.3680043.10051,213.56364,216.71419,25.13455,47.92717,138
6,1.2.826.0.1.3680043.10051,209.00000,214.57591,36.06131,51.56834,139
7,1.2.826.0.1.3680043.10051,204.25455,212.24727,47.47636,57.71635,140
8,1.2.826.0.1.3680043.10051,205.00000,213.87342,45.00000,59.12658,141
9,1.2.826.0.1.3680043.10051,205.18545,214.76814,42.89892,63.43364,142


In [9]:
df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
df = df[df['patient_overall']==0]
df

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
7,1.2.826.0.1.3680043.17364,0,0,0,0,0,0,0,0
8,1.2.826.0.1.3680043.849,0,0,0,0,0,0,0,0
12,1.2.826.0.1.3680043.24946,0,0,0,0,0,0,0,0
13,1.2.826.0.1.3680043.9290,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2011,1.2.826.0.1.3680043.32263,0,0,0,0,0,0,0,0
2012,1.2.826.0.1.3680043.6398,0,0,0,0,0,0,0,0
2013,1.2.826.0.1.3680043.31621,0,0,0,0,0,0,0,0
2016,1.2.826.0.1.3680043.14341,0,0,0,0,0,0,0,0


In [10]:
a_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
a_df = a_df.set_index(a_df['StudyInstanceUID'])
a_df

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
StudyInstanceUID,,,,,,,,,
1.2.826.0.1.3680043.6200,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1.2.826.0.1.3680043.27262,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
1.2.826.0.1.3680043.21561,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
1.2.826.0.1.3680043.12351,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
1.2.826.0.1.3680043.1363,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1.2.826.0.1.3680043.21684,1.2.826.0.1.3680043.21684,1,0,1,0,0,0,1,1
1.2.826.0.1.3680043.4786,1.2.826.0.1.3680043.4786,1,0,0,0,0,0,0,1
1.2.826.0.1.3680043.14341,1.2.826.0.1.3680043.14341,0,0,0,0,0,0,0,0


In [11]:
path = "../input/rsna-2022-cervical-spine-fracture-detection/train_images/"

In [12]:
import random

def my_distribution(maxx):
    THRESHOLD = [0.05, 0.95]
    u = random.random()
    if u <= THRESHOLD[0]:
        return random.randint(1, maxx//5)
    elif u <= THRESHOLD[1]:
        return random.randint( (maxx//5),  ((4*maxx)//5))
    else:
        return random.randint(((4*maxx)//5), maxx)

In [31]:
t = 5
n_all = t*7217
n_80 = ((t*7217)*(8))//10
deviation = n_all//1058 
np_list = np.arange(n_all)
random_list = np.random.choice(np_list,n_all,replace=False)

def data_gen():
    for i in random_list[:n_80]:
        if i%t!=0:
            try:     
                img = window(load_image(path+df.iloc[i//int(deviation)]["StudyInstanceUID"]+"/"+str(my_distribution(len(os.listdir(path+df.iloc[i//int(deviation)]["StudyInstanceUID"]))))+".dcm"))
                yield ({"input_1": channeling(img)},
                {'co': 0, 
                                        'c1': 0,
                                        'c2': 0,
                                        'c3': 0,
                                        'c4': 0,
                                        'c5': 0,
                                        'c6': 0,
                                        'c7': 0,},
                      {'co': 7, 
                                    'c1': 1,
                                    'c2': 1,
                                    'c3': 1,
                                    'c4': 1,
                                    'c5': 1,
                                    'c6': 1,
                                    'c7': 1})
            except:
                continue

        else:         
            img = window(load_image(path+bb_df.iloc[i//t]["StudyInstanceUID"]+"/"+str(bb_df.iloc[i//t]["slice_number"])+".dcm"))
            yield ({"input_1": channeling(img)},
                {'co': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["patient_overall"], 
                                        'c1': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["C1"],
                                        'c2': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["C2"],
                                        'c3': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["C3"],
                                        'c4': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["C4"],
                                        'c5': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["C5"],
                                        'c6': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["C6"],
                                        'c7': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["C7"]},
                      {'co': 14, 
                                    'c1': 2,
                                    'c2': 2,
                                    'c3': 2,
                                    'c4': 2,
                                    'c5': 2,
                                    'c6': 2,
                                    'c7': 2})
        
        
def test_gen():
    for i in random_list[n_80:]:
        if i%t!=0:
            try:     
                img = window(load_image(path+df.iloc[i//int(deviation)]["StudyInstanceUID"]+"/"+str(my_distribution(len(os.listdir(path+df.iloc[i//int(deviation)]["StudyInstanceUID"]))))+".dcm"))
                yield ({"input_1": channeling(img)},
                {'co': 0, 
                                        'c1': 0,
                                        'c2': 0,
                                        'c3': 0,
                                        'c4': 0,
                                        'c5': 0,
                                        'c6': 0,
                                        'c7': 0,},
                      {'co': 7, 
                                    'c1': 1,
                                    'c2': 1,
                                    'c3': 1,
                                    'c4': 1,
                                    'c5': 1,
                                    'c6': 1,
                                    'c7': 1})
            except:
                continue

        else:         
            img = window(load_image(path+bb_df.iloc[i//t]["StudyInstanceUID"]+"/"+str(bb_df.iloc[i//t]["slice_number"])+".dcm"))
            yield ({"input_1": channeling(img)},
                    {'co': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["patient_overall"], 
                                        'c1': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["C1"],
                                        'c2': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["C2"],
                                        'c3': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["C3"],
                                        'c4': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["C4"],
                                        'c5': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["C5"],
                                        'c6': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["C6"],
                                        'c7': a_df.loc[bb_df.iloc[int(i//t)]["StudyInstanceUID"]]["C7"]},
                      {'co': 14, 
                                    'c1': 2,
                                    'c2': 2,
                                    'c3': 2,
                                    'c4': 2,
                                    'c5': 2,
                                    'c6': 2,
                                    'c7': 2})



In [32]:
dataset = tf.data.Dataset.from_generator(
     data_gen,
     ({"input_1":tf.float32}, 
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8},
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8}),
    ({"input_1":tf.TensorShape([160,160,3])}, 
     {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])},
    {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])})
)

testset = tf.data.Dataset.from_generator(
     test_gen,
     ({"input_1":tf.float32}, 
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8},
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8}),
    ({"input_1":tf.TensorShape([160,160,3])}, 
     {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])},
    {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])})
)

In [33]:
augment = tf.keras.Sequential([
  keras.layers.RandomFlip("horizontal_and_vertical"),
  keras.layers.RandomRotation(0.4),
  keras.layers.GaussianNoise(0.2),
  keras.layers.RandomFlip(mode="horizontal_and_vertical", seed=5),
  keras.layers.RandomContrast(0.6, seed=2),
  keras.layers.RandomZoom(
    (-0.1, 0.1),
    (-0.1, 0.1),
    fill_mode='reflect',
    interpolation='bilinear',
    seed=3)
])
    

In [34]:
dataset = dataset.map(lambda x, y, z: (augment(x['input_1']), y, z), 
              num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(30)
dataset = dataset.prefetch(2)


testset = testset.batch(30)
testset = testset.prefetch(2)

In [36]:
res = tf.keras.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    pooling='avg',
    input_shape=(160,160,3),
)

inputs = keras.Input((160, 160, 3), name = "input_1")
x = res(inputs)
outputo = keras.layers.Dense(1 ,activation = 'sigmoid', name="co")(x)
output1 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c1")(x)
output2 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c2")(x)
output3 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c3")(x)
output4 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c4")(x)
output5 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c5")(x)
output6 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c6")(x)
output7 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c7")(x)
                                               
model = keras.models.Model(inputs=inputs, outputs=[outputo,output1,output2,output3,output4,output5,output6,output7])


74850304/74836368 [==============================] - 0s 0us/step


In [37]:
opt = tfa.optimizers.AdamW(
    weight_decay=2e-4,
    learning_rate = 0.0003,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'AdamW',
)
loss = tf.keras.losses.BinaryCrossentropy(
    label_smoothing=0.1, 
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)
metrics = [tf.keras.metrics.AUC()]
model.compile(optimizer=opt,
              loss={"co": loss, 
                   "c1": loss,
                   "c2": loss,
                   "c3": loss,
                   "c4": loss,
                   "c5": loss,
                   "c6": loss,
                   "c7": loss,},
                metrics={"co": metrics, 
                   "c1": metrics,
                   "c2": metrics,
                   "c3": metrics,
                   "c4": metrics,
                   "c5": metrics,
                   "c6": metrics,
                   "c7": metrics,})

In [ ]:
!mkdir best_weights

In [ ]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath = './best_weights/model',
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
)

In [38]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
densenet201 (Functional)        (None, 1920)         18321984    input_1[0][0]                    
__________________________________________________________________________________________________
co (Dense)                      (None, 1)            1921        densenet201[0][0]                
__________________________________________________________________________________________________
c1 (Dense)                      (None, 1)            1921        densenet201[0][0]                
______________________________________________________________________________________________

In [41]:
history = model.fit(dataset, validation_data = testset, epochs=15, callbacks=[model_checkpoint])

Epoch 1/4
960/960 [==============================] - 628s 654ms/step - loss: 7.0722 - co_loss: 4.0824 - c1_loss: 0.3811 - c2_loss: 0.4138 - c3_loss: 0.3968 - c4_loss: 0.4169 - c5_loss: 0.4626 - c6_loss: 0.4652 - c7_loss: 0.4534 - c7_c6_c5_c4_c3_c2_c1_co_auc: 0.8971 - val_loss: 9.1794 - val_co_loss: 5.3690 - val_c1_loss: 0.4219 - val_c2_loss: 0.5237 - val_c3_loss: 0.4308 - val_c4_loss: 0.5284 - val_c5_loss: 0.6804 - val_c6_loss: 0.6826 - val_c7_loss: 0.5425 - val_c7_c6_c5_c4_c3_c2_c1_co_auc: 0.7916
Epoch 2/4
960/960 [==============================] - 611s 636ms/step - loss: 6.8729 - co_loss: 3.9551 - c1_loss: 0.3711 - c2_loss: 0.4047 - c3_loss: 0.3847 - c4_loss: 0.4070 - c5_loss: 0.4518 - c6_loss: 0.4534 - c7_loss: 0.4450 - c7_c6_c5_c4_c3_c2_c1_co_auc: 0.9114 - val_loss: 7.1471 - val_co_loss: 4.1367 - val_c1_loss: 0.3747 - val_c2_loss: 0.4058 - val_c3_loss: 0.3919 - val_c4_loss: 0.4364 - val_c5_loss: 0.4596 - val_c6_loss: 0.4884 - val_c7_loss: 0.4537 - val_c7_c6_c5_c4_c3_c2_c1_co_auc: 0

In [ ]:
!mkdir final_model

In [ ]:
# model.load_weights('./best_weights/model').expect_partial()
model.save("./final_model/model")